# scrape

In [1]:
import os
import time
import datetime as dt
from datetime import datetime
from multiprocessing import Pool
from recipe_scrapers import scrap_me

from time import sleep, time
from random import randint, choice
import requests
import urllib
from bs4 import BeautifulSoup
import csv
import re

In [2]:
# OS
NOW           = dt.datetime.now()
FILE_NAME     = 'recipe.csv'
DATASET_FOLDER = 'input/csv_files/'
IMGS_FOLDER  = 'input/images/search_thumbnails/'

# Chefkoch.de Seite
CHEFKOCH_URL  = 'http://www.chefkoch.de'
START_URL     = 'http://www.chefkoch.de/rs/s'
CATEGORY      = '/Rezepte.html'

In [3]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

def random_headers():
    return {'User-Agent': choice(desktop_agents),'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

In [4]:
category_url = START_URL + '0o3' + CATEGORY

def _get_html(url):
    page = ''
    while page == '':
        try:
            page = requests.get(url, headers=random_headers())
        except:
            print('Connection refused')
            time.sleep(10)
            continue
    return page.text

def _get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
   #print(soup.prettify())
    total_pages = soup.find('div', class_='ck-pagination qa-pagination').find('a', class_='qa-pagination-pagelink-last').text
    print(total_pages)
    #return int(total_pages)


#print('Total pages: ', total_pages)

In [5]:
def _write_to_recipes(data):
    path = FILE_NAME
    with open(path, 'a') as f:
        writer = csv.writer(f)
        try:
            writer.writerow((data['recipe_id'],
                            data['recipe_name'],
                            data['average_rating'],
                            data['stars_shown'],
                            data['votes'],
                            data['difficulty'],
                            data['preparation_time'],
                            data['date'],
                            data['link'],
                            data['has_picture']))
        except:
            pass

        
def _get_images(html):
    soup = BeautifulSoup(html, 'lxml')
    

def _get_links(html):
    urlArray = []
    soup = BeautifulSoup(html, 'lxml')
    links = soup.find_all('div', class_='archiverecipe')#get all links
    for individual in links:
        href = individual.find('a')['href']
        urlArray.append(href)
        scrap_me = scrap_me(href)

        scrap_me.title()
        scrap_me.total_time()
        scrap_me.ingredients()
        scrap_me.instructions()
        break

def _get_data(html): 
    soup = BeautifulSoup(html, 'lxml')
    #name
    try:
        name = soup.find('h1', class_='typog-headline4___Alhic').find('span').text
    except:
        name = ''
    # difficulty
    try:
        difficulty = soup.find_all('div', class_='typog-bodycopySmall___DIQrd value___1j1Hl')[4].find('div').text
    except:
        difficulty = ''
    try:
        preptime = soup.find_all('div', class_='typog-bodycopySmall___DIQrd value___1j1Hl')[1].find('div').text
    except:
        preptime = ''
    data = {'recipe_id' : '',
        'recipe_name' : name,
        'average_rating': '',
        'stars_shown' : '',
        'votes' : '',
        'difficulty' : difficulty,
        'preparation_time' : preptime,
        'has_picture' : '',
        'date' : '',
        'link' : ''}
    _write_to_recipes(data)

_get_links(_get_html('https://www.101cookbooks.com/archives.html'))

UnboundLocalError: local variable 'scrap_me' referenced before assignment

In [6]:
start_time = time()


'''
all_links = _get_links(_get_html('https://www.weightwatchers.com/us/recipe'))
(_get_html(all_links[5]))

for link in range(3,len(all_links)):
    sub = _get_links(_get_html(all_links[link]))
    for individual in sub:
        _get_data(_get_html(individual))
    #print(len(_get_images(_get_html(link))))
'''

print("--- %s seconds ---" % (time() - start_time))

--- 9.29832458496e-05 seconds ---


In [237]:
import pandas as pd
output = pd.read_csv('recipe.csv', header=None)
print(output.head())

    0                                                  1   2   3   4     5  \
0 NaN                             Tex-Mex Scrambled Eggs NaN NaN NaN  Easy   
1 NaN  Baked Eggs with Delicata Squash, Spinach and Feta NaN NaN NaN  Easy   
2 NaN             Greek Yogurt with Warm Blueberry Sauce NaN NaN NaN  Easy   
3 NaN                   Individual Shakshouka Baked Eggs NaN NaN NaN  Easy   
4 NaN           Peanutty Vanilla Yogurt with Blueberries NaN NaN NaN   NaN   

        6   7   8   9  
0  18 min NaN NaN NaN  
1  25 min NaN NaN NaN  
2   8 min NaN NaN NaN  
3  20 min NaN NaN NaN  
4     NaN NaN NaN NaN  


In [241]:
output.columns = ['recipe_id', 'recipe_name', 'average_rating',
                        'stars_shown', 'votes', 'difficulty', 'preparation_time',
                        'date', 'link', 'has_picture']

In [242]:
output.head()

,recipe_id,recipe_name,average_rating,stars_shown,votes,difficulty,preparation_time,date,link,has_picture
0,NaN,Tex-Mex Scrambled Eggs,NaN,NaN,NaN,Easy,18 min,NaN,NaN,NaN
1,NaN,"Baked Eggs with Delicata Squash, Spinach and Feta",NaN,NaN,NaN,Easy,25 min,NaN,NaN,NaN
2,NaN,Greek Yogurt with Warm Blueberry Sauce,NaN,NaN,NaN,Easy,8 min,NaN,NaN,NaN
3,NaN,Individual Shakshouka Baked Eggs,NaN,NaN,NaN,Easy,20 min,NaN,NaN,NaN
4,NaN,Peanutty Vanilla Yogurt with Blueberries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import os
path = 'input/images/search_thumbnails/'
files = os.listdir(path)
i = 1
for file in files:
    filename, file_extension = os.path.splitext(file)
    os.rename(os.path.join(path, file), os.path.join(path, filename + '-0' + file_extension))
    print('renamed: ', i)
    i = i+1

# Weiter gehts mit Teil 2: 02_rezepte_details.ipynb